In [1]:
from openfisca_us import Microsimulation
from openfisca_us.api import *
from openfisca_us.entities import *
from openfisca_us.tools.general import *
from openfisca_us import reforms
import numpy as np

In [2]:
# Calculate poverty rate
base = Microsimulation(year=2020)
base.calc("in_poverty", map_to="person").mean()

0.10159000907591269

In [15]:
class basic_income(Variable):
    value_type = float
    entity = TaxUnit
    definition_period = YEAR
    label = "Basic income"

    def formula(tax_unit, period, parameters):
        # Extract FPG parameter
        fpg_params = parameters(period).poverty.fpg
        # Includes first_person and additional_person
        # Extract tax unit number of people
        nb_people = tax_unit.nb_persons()
        # Calculate FPG
        fpg = fpg_params.first_person.contiguous_US + (nb_people - 1) * fpg_params.additional_person.contiguous_US
        # Extract taxable income
        taxable_income = tax_unit("taxable_income", period)
        # Calculate basic income phased out at 50%
        return np.maximum(fpg - taxable_income * 0.5, 0)
        # TODO: Make 50% a parameter

In [27]:
class taxable_income(Variable):
    value_type = float
    entity = TaxUnit
    definition_period = YEAR

In [28]:
class SPM_unit_net_income(Variable):
    value_type = float
    entity = SPMUnit
    label = u"SPM unit net income"
    definition_period = YEAR

    def formula(spm_unit, period):
        INCOME_COMPONENTS = [
            "SPM_unit_total_income",
            "SPM_unit_SNAP",
            "SPM_unit_capped_housing_subsidy",
            "SPM_unit_school_lunch_subsidy",
            "SPM_unit_energy_subsidy",
            "SPM_unit_WIC",
        ]
        EXPENSE_COMPONENTS = [
            "SPM_unit_FICA",
            "SPM_unit_federal_tax",
            "SPM_unit_state_tax",
            "SPM_unit_capped_work_childcare_expenses",
            "SPM_unit_medical_expenses",
        ]
        income = add(spm_unit, period, *INCOME_COMPONENTS)
        expense = add(spm_unit, period, *EXPENSE_COMPONENTS)
        basic_income = sum_contained_tax_units("basic_income", spm_unit, period)
        return income - expense + basic_income

In [29]:
class reform(Reform):
    def apply(self):
        self.update_variable(SPM_unit_net_income)
        self.add_variable(basic_income)
        self.add_variable(taxable_income)

# reform = reforms.restructure(SPM_unit_net_income), reforms.new_variable(basic_income)
reform_sim = Microsimulation(reform, year=2020)
from openfisca_us_data import RawCPS
taxable_income_values = RawCPS.load(2020, "tax_unit").TAX_INC.values
reform_sim.simulation.set_input("taxable_income", 2020, taxable_income_values)

In [34]:
reform_sim.df(["SPM_unit_net_income", "taxable_income", "basic_income"])

,SPM_unit_net_income,taxable_income,basic_income,weight
0,90113.750000,98944.0,0.0,1552.900024
1,62220.000000,14589.0,5465.5,990.489990
2,43478.000000,24720.0,4880.0,1505.270020
3,27279.060547,0.0,12760.0,1430.699951
4,89555.000000,0.0,12760.0,1133.369995
...,...,...,...,...
82193,NaN,0.0,21720.0,NaN
82194,NaN,73783.0,0.0,NaN
82195,NaN,0.0,26200.0,NaN
82196,NaN,0.0,12760.0,NaN


In [33]:
gain = reform_sim.calc("SPM_unit_net_income") - base.calc("SPM_unit_net_income")
gain.groupby(base.calc("SPM_unit_net_income").decile_rank()).mean()

1.0      6950.139729
2.0      7377.274381
3.0      8676.671452
4.0      9226.481163
5.0      9980.518635
6.0     10343.661848
7.0     11180.844588
8.0     11710.768067
9.0     12154.836279
10.0    12263.114358
dtype: float64

In [30]:
reform_sim.calc("SPM_unit_net_income").mean() - base.calc("SPM_unit_net_income").mean()

9986.52237030775

In [31]:
reform_sim.calc("basic_income").mean()

7837.700207688253